### **Data Preprocessing**



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder



In [2]:
# Load Data
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y_df = pd.read_csv('../Data/WISDM_y.csv')
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [3]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
x_train_tensor = torch.tensor(X_train, dtype=torch.float32)
x_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)



In [4]:
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, n_steps):
        super(LSTMNet, self).__init__()
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.lstm1 = nn.LSTM(input_size, hidden_size1, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(hidden_size1, hidden_size2, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(1, x.size(0), self.hidden_size1).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size1).to(x.device)
        
        # First LSTM layer
        out, _ = self.lstm1(x, (h0, c0))
        out = self.dropout1(out)
        
        # Second LSTM layer
        h1 = torch.zeros(1, x.size(0), self.hidden_size2).to(x.device)
        c1 = torch.zeros(1, x.size(0), self.hidden_size2).to(x.device)
        out, _ = self.lstm2(out, (h1, c1))
        out = self.dropout2(out)

        # Dense layer
        out = self.fc(out[:, -1, :]) # Taking the last time step
        return out


In [5]:
# Parameters
input_size = 3  # Number of features
hidden_size1 = 50
hidden_size2 = 25
output_size = 6
n_steps = 16

# Create the model
model = LSTMNet(input_size, hidden_size1, hidden_size2, output_size, n_steps)

# Training setup (for demonstration)
# Define your dataset here
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.int64))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [6]:
def train(model, train_loader, loss_fn, optimizer, epochs=0):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(y_pred.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        avg_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

train(model, train_loader, loss_fn, optimizer, epochs=20)


Epoch [1/20], Loss: 0.5917, Accuracy: 79.39%
Epoch [2/20], Loss: 0.3647, Accuracy: 87.25%
Epoch [3/20], Loss: 0.2918, Accuracy: 90.15%
Epoch [4/20], Loss: 0.2531, Accuracy: 91.54%
Epoch [5/20], Loss: 0.2271, Accuracy: 92.47%
Epoch [6/20], Loss: 0.2097, Accuracy: 93.16%
Epoch [7/20], Loss: 0.1971, Accuracy: 93.55%
Epoch [8/20], Loss: 0.1855, Accuracy: 93.85%
Epoch [9/20], Loss: 0.1755, Accuracy: 94.33%
Epoch [10/20], Loss: 0.1681, Accuracy: 94.54%
Epoch [11/20], Loss: 0.1625, Accuracy: 94.69%
Epoch [12/20], Loss: 0.1569, Accuracy: 94.93%
Epoch [13/20], Loss: 0.1486, Accuracy: 95.17%
Epoch [14/20], Loss: 0.1452, Accuracy: 95.29%
Epoch [15/20], Loss: 0.1409, Accuracy: 95.48%
Epoch [16/20], Loss: 0.1362, Accuracy: 95.54%
Epoch [17/20], Loss: 0.1346, Accuracy: 95.61%
Epoch [18/20], Loss: 0.1297, Accuracy: 95.80%
Epoch [19/20], Loss: 0.1260, Accuracy: 95.93%
Epoch [20/20], Loss: 0.1232, Accuracy: 96.02%


In [7]:
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.int64))
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
def evaluate(model, test_loader, loss_fn):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            total_loss += loss_fn(y_pred, y_batch).item()
            _, predicted = torch.max(y_pred.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total
    return avg_loss, accuracy

avg_loss, accuracy = evaluate(model, test_loader, loss_fn)
print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}")



Test Loss: 0.1601, Test Accuracy: 0.9506


In [8]:
model_path = "/Users/sandeep/Desktop/BUCourses/Project/saved_models/Pytorch/lstm_base.pth"
torch.save(model.state_dict(), model_path)

### Convert to Coreml

In [9]:
import coremltools as ct
example_input = torch.rand(1,16, 3) 

model.eval()
traced_model = torch.jit.trace(model, example_input)
out = traced_model(example_input)

# Convert to Core ML program using the Unified Conversion API.
mlp_coreml_model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

# Save the converted model.
mlp_coreml_model.save("lstm.mlpackage")

scikit-learn version 1.4.0 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.
XGBoost version 1.7.6 has not been tested with coremltools. You may run into unexpected errors. XGBoost 1.4.2 is the most recent version that has been tested.
TensorFlow version 2.15.0 has not been tested with coremltools. You may run into unexpected errors. TensorFlow 2.12.0 is the most recent version that has been tested.
Running MIL backend_mlprogram pipeline: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 12409.18 passes/s]


### Define Coreml Metrics

In [14]:
from pathlib import Path

def coreml_metrics(model_name, X_test, y_test, model_path):
    predictions = []
    for id in range(len(X_test)):
        X_test_new = np.expand_dims(X_test[id], axis=0)
        output_dict = model_name.predict({'x': X_test_new})
        pred_class = np.argmax(output_dict['linear_0'])
        predictions.append(pred_class)
    
    accuracy = np.sum(predictions == y_test) / len(predictions)
    print("Accuracy:", accuracy)
    
    model_file = Path(model_path)
    
    # Size in bytes
    model_size_bytes = model_file.stat().st_size
    
    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

In [11]:
import coremltools as ct
import coremltools.optimize.coreml as cto

lstm_coreml_model = ct.models.MLModel("lstm.mlpackage")

In [15]:
model_name = lstm_coreml_model
model_path = "lstm.mlpackage"

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9505372427608815
Size of the model: 0.12 KB


### 8-bit Quantization coreml

In [16]:
import coremltools.optimize.coreml as cto

op_config = cto.OpLinearQuantizerConfig(mode="linear_symmetric", weight_threshold=512)
config = cto.OptimizationConfig(global_config=op_config)

lstm_compressed_8_bit_model = cto.linear_quantize_weights(lstm_coreml_model, config=config)


Running compression pass linear_quantize_weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 2779.86 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 7130.14 passes/s]


In [17]:
lstm_compressed_8_bit_model.save("lstm_8bitQuantized_mlmodel.mlpackage")

In [18]:
model_name = lstm_compressed_8_bit_model
model_path = "lstm_8bitQuantized_mlmodel.mlpackage"

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9505372427608815
Size of the model: 0.12 KB
